Projeto desenvolvido no âmbito do módulo de Técnicas de Programação I, na trilha de Engenharia de dados do programa Santander Coders 2023, em parceria com a Ada Tech.

Projeto elaborado por:
- Andreza Lima
- Danilo Freitas
- Luis Felipe Gilisk
- Patrick Leite

# Descrição do projeto


Você trabalha em uma consultoria de dados que foi contratada para realizar a distribuição de materiais didáticos nas escolas da cidade do Rio de Janeiro. Sua missão é realizar tratamentos nos dados de acordo com as normas de padrão definidas pelo cliente e encontrar qual a melhor rota que um caminhão deve realizar para entregar os materiais didáticos de forma a otimizar o seu percurso. 

Para esse projeto você recebeu três arquivos:
- escolas.csv: contém os dados das escolas
- subprefeituras.csv: contém dados de quais bairros pertem a cada subprefeitura
- material_didatico.csv: contém a quantidade de material didático que cada escola deve receber

Como produto final, você deve entregar:
- um arquivo csv contendo os dados no padrão especificado com as escolas ordenadas em que os materiais didáticos devem ser entregues e com as colunas contendo id da escola, nome da escola, tipo da escola (EM, CIEP ou colégio), logradouro da entrega, número, bairro, subprefeitura, latitude, longitude e quantidade de material didático que deve ser entregue;
- um arquivo csv com a quantidade total de material escolar por subprefeitura para que sejam contabilizados os custos por subprefeitura

Como padrão dos dados, considere:
- nome das colunas em snake_case
- strings não devem conter acentos
- todas as strings devem estar em maiúsculo
- padronização do nome dos logradouros sem abreviação (Ex: R. deve estar como Rua)
- latitude e longitude devem conter apenas 5 casas decimais
- os ids da escola devem todos ser strings com 3 caracteres (Ex: '024')

Desafio:

Entregar um plot com a representação da melhor rota que você encontrou, por exemplo:
<img src='rota.png' width=600>


_______


# Proposta de resolução

## Importando pacotes

In [1]:
## Executar apenas se for preciso instalar pacotes
# !pip install pandas
# !pip install unidecode

In [2]:
import pandas as pd
import unidecode 

## Funções auxiliares

In [3]:
# Função para tratamento de colunas strings (remove acentos, coloca em maiúsculo, remove abreviações)
def trata_string(df,*colunas):
    """
    Recebe um data frame e os nomes das colunas que devem ter seu conteúdo transformado
    Retorna o mesmo data frame com as colunas transformadas
    """
    for coluna in colunas:
        df[coluna] = [unidecode.unidecode(x) for x in df[coluna].str.upper() ]
    return df

## Tratamento de escolas.csv

### Conhecendo os dados

In [4]:
# Leitura do arquivo
escolas = pd.read_csv("escolas.csv")

In [5]:
# Verifica a estrutura
escolas.head()

,id,Escolas_Postos,BAIRRO,ENDEREÇO,lat,lon
0,178,CENTRO INTEGRADO DE EDUCAÇÃO PÚBLICA HENFIL,CAJU,Rua Carlos Seidl S/Nº,"-22,880888","-43,225326"
1,634,EM ALICE DO AMARAL PEIXOTO,BENFICA,Rua Ébano 187,"-22,889574","-43,236202"
2,483,EM CELESTINO SILVA,CENTRO,"R. do Lavradio, 56","-22,909293","-43,183579"
3,476,ESCOLA MUNICIPAL FLORIANO PEIXOTO,SÃO CRISTÓVÃO,Praça Argentina 20,"-22,897629","-43,227456"
4,132,EM PEREIRA PASSOS,RIO COMPRIDO,Praça Condessa Paulo de Frontin 45,"-22,924412","-43,208579"


In [6]:
# verifica informações mais detalhadas do data frame
escolas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152 entries, 0 to 151
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              152 non-null    int64 
 1   Escolas_Postos  152 non-null    object
 2   BAIRRO          152 non-null    object
 3   ENDEREÇO        152 non-null    object
 4   lat             152 non-null    object
 5   lon             152 non-null    object
dtypes: int64(1), object(5)
memory usage: 7.2+ KB


Destes resultados, é possível verificar que:
- os nomes das colunas não estão em formato snake case
- existe um ç nos nomes das colunas
- o dataset contém 152 escolas
- nenhum campo contém nulos
- o campo id tem tipologia de número inteiro, quando pelas instruções deveria ser string com tamanho padronizado em 3
- os campos escolas_postos, bairro e endereço precisam ser tratados para:
    - não conter acentos
    - estarem todas em maiúsculo
    - remover abreviação
- os campos latitude e longitude devem ser tratados para conter apenas 5 casas decimais. (TRANFORMAR EM FLOAT?)

A partir desta constatação, serão executadas ações para corrigir os pontos identificados acima.

### Corrigindo nomes das colunas

In [7]:
## Renomeia os nomes das colunas para snake case

# transforma os nomes das colunas em minúsculos 
escolas.columns = escolas.columns.str.lower()

# remove espaços antes e depois dos nomes das colunas, se houver
escolas.columns = escolas.columns.str.strip()

# remove os acentos dos nomes das colunas
escolas.columns = [unidecode.unidecode(x) for x in escolas.columns ]

In [8]:
# verifica a nova estrutura
escolas.head()

,id,escolas_postos,bairro,endereco,lat,lon
0,178,CENTRO INTEGRADO DE EDUCAÇÃO PÚBLICA HENFIL,CAJU,Rua Carlos Seidl S/Nº,"-22,880888","-43,225326"
1,634,EM ALICE DO AMARAL PEIXOTO,BENFICA,Rua Ébano 187,"-22,889574","-43,236202"
2,483,EM CELESTINO SILVA,CENTRO,"R. do Lavradio, 56","-22,909293","-43,183579"
3,476,ESCOLA MUNICIPAL FLORIANO PEIXOTO,SÃO CRISTÓVÃO,Praça Argentina 20,"-22,897629","-43,227456"
4,132,EM PEREIRA PASSOS,RIO COMPRIDO,Praça Condessa Paulo de Frontin 45,"-22,924412","-43,208579"


### Corrigindo Ids

In [9]:
# verifica id
escolas.id.describe()

count    152.000000
mean     342.638158
std      203.307667
min       11.000000
25%      160.000000
50%      346.500000
75%      520.750000
max      690.000000
Name: id, dtype: float64

Verifica-se que existem inteiros com menos de 3 caracteres, então além de transformar a coluna em string, será necessário incluir zeros à esquerda, quando necessário

In [10]:
# tranforma coluna id em string:
escolas["id"] = escolas.id.astype(str)

In [11]:
escolas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152 entries, 0 to 151
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              152 non-null    object
 1   escolas_postos  152 non-null    object
 2   bairro          152 non-null    object
 3   endereco        152 non-null    object
 4   lat             152 non-null    object
 5   lon             152 non-null    object
dtypes: object(6)
memory usage: 7.2+ KB


Como pandas usa object dtype para armazenar strings, resta agora apenas adicionar os zeros.

In [12]:
# Inclui zeros até que a string tenha tamanho 3
escolas["id"] = escolas.id.str.zfill(3)

In [13]:
# Checando os novos ids
escolas.id.unique()

array(['178', '634', '483', '476', '132', '017', '055', '490', '600',
       '089', '387', '331', '450', '558', '413', '540', '309', '268',
       '523', '035', '011', '226', '638', '126', '180', '163', '361',
       '280', '060', '587', '653', '660', '382', '217', '378', '429',
       '025', '373', '624', '157', '020', '117', '502', '351', '135',
       '388', '440', '259', '422', '041', '537', '230', '379', '146',
       '566', '498', '663', '121', '161', '263', '118', '140', '260',
       '069', '090', '274', '676', '390', '608', '589', '256', '091',
       '248', '623', '603', '673', '520', '412', '031', '514', '023',
       '415', '016', '570', '528', '414', '690', '202', '550', '211',
       '206', '492', '466', '636', '527', '586', '080', '398', '175',
       '308', '576', '495', '641', '689', '171', '477', '687', '568',
       '399', '255', '262', '616', '320', '144', '534', '342', '052',
       '139', '323', '573', '560', '246', '319', '584', '311', '316',
       '497', '297',

### Tratando colunas do tipo string

In [14]:
trata_string(escolas,"escolas_postos","bairro","endereco")

,id,escolas_postos,bairro,endereco,lat,lon
0,178,CENTRO INTEGRADO DE EDUCACAO PUBLICA HENFIL,CAJU,RUA CARLOS SEIDL S/No,"-22,880888","-43,225326"
1,634,EM ALICE DO AMARAL PEIXOTO,BENFICA,RUA EBANO 187,"-22,889574","-43,236202"
2,483,EM CELESTINO SILVA,CENTRO,"R. DO LAVRADIO, 56","-22,909293","-43,183579"
3,476,ESCOLA MUNICIPAL FLORIANO PEIXOTO,SAO CRISTOVAO,PRACA ARGENTINA 20,"-22,897629","-43,227456"
4,132,EM PEREIRA PASSOS,RIO COMPRIDO,PRACA CONDESSA PAULO DE FRONTIN 45,"-22,924412","-43,208579"
...,...,...,...,...,...,...
147,474,EM ENGENHEIRO GASTAO RANGEL,GUARATIBA,ESTRADA DO MAGARCA 9.183,"-22,98046","-43,643545"
148,301,EM JONATAS SERRANO,GUARATIBA,"ESTRADA DO MATO ALTO, S/No","-22,953163","-43,577409"
149,215,E.M. NARCISA AMALIA,ILHA DE GUARATIBA,ESTRADA TEODORETO DE CAMARGO S/N.o,"-23,009084","-43,537582"
150,606,EM PROFESSOR CASTILHO,ILHA DE GUARATIBA,CAMINHO DA MATRIZ 4406,"-22,994124","-43,593683"


Para padronizar as siglas, o conhecimento da área de negócio seria o mais poderoso insight para resolver todas as ocorrências. Mas na ausência deste conhecimento, será necessário mais algumas investigações.

In [15]:
# Verificando as entradas distintas para escolas_postos
escolas.escolas_postos.unique()

array(['CENTRO INTEGRADO DE EDUCACAO PUBLICA HENFIL',
       'EM ALICE DO AMARAL PEIXOTO', 'EM CELESTINO SILVA',
       'ESCOLA MUNICIPAL FLORIANO PEIXOTO', 'EM PEREIRA PASSOS',
       'E.M PRESIDENTE JOSE LINHARES', 'EM SANTA CATARINA',
       'E.M TIA CIATA', 'EM URUGUAI', 'EM MARIO CLAUDIO',
       'CIEP PRESIDENTE AGOSTINHO NETO', 'CIEP TANCREDO NEVES',
       'EM ALBERTO BARTH', 'EM CICERO PENA', 'EM COCIO BARCELOS',
       'EM HENRIQUE DODSWORTH', 'EM MEXICO',
       'ESCOLA MUNICIPAL MINAS GERAIS', 'EM PRESIDENTE DA COSTA E SILVA',
       'EM SENADOR CORREA', 'CIEP SAMUEL WEINER', 'EM AFRANIO PEIXOTO',
       'E.M AZEVEDO SODRE', 'EM DUQUE DE CAXIAS',
       'ESCOLA MUNICIPAL EQUADOR', 'EM LAUDIMIA TROTTA',
       'EM PRUDENTE DE MORAES',
       'ESCOLA MUNICIPAL GENERAL HUMBERTO DE SOUZA MELLO',
       'ESCOLA MUNICIPAL BARAO HOMEM DE MELO', 'EM ALMIRANTE TAMANDARE',
       'EM LUCIA MIGUEL PEREIRA', 'E.M NOSSA SENHORA DA POMPEIA',
       'EM SANTOS ANJOS', 'EM ALAGOAS', 'EM BR

In [16]:
# Verificando as entradas distintas para bairro
escolas.bairro.unique()

array(['CAJU', 'BENFICA', 'CENTRO', 'SAO CRISTOVAO', 'RIO COMPRIDO',
       'IPANEMA', 'SANTA TERESA', 'HUMAITA', 'CATETE', 'FLAMENGO',
       'COPACABANA', 'BOTAFOGO', 'URCA', 'LARANJEIRAS', 'TIJUCA',
       'ANDARAI', 'PRACA DA BANDEIRA', 'GRAJAU', 'VILA ISABEL',
       'MARACANA/ VILA ISABEL', 'MARACANA/ TIJUCA', 'VIDIGAL',
       'SAO CONRADO', 'TODOS OS SANTOS', 'LEBLON', 'PILARES',
       'AGUA SANTA', 'PIEDADE', 'RIACHUELO', 'MEIER', 'ENGENHO DE DENTRO',
       'ENGENHO NOVO', 'JACARE', 'DEL CASTILHO', 'INHAUMA',
       'PARADA DE LUCAS', 'MARIA DA GRACA', 'MANGUINHOS',
       'JARDIM AMERICA', 'PENHA CIRCULAR', 'OLARIA', 'CAMPO GRANDE',
       'PENHA', 'RAMOS', 'VIGARIO GERAL', 'BONSUCESSO', 'MADUREIRA',
       'ACARI', 'CACUIA', 'JARDIM GUANABARA', 'PARQUE COLUMBIA', 'PAVUNA',
       'ANCHIETA', 'GUADALUPE', 'TURIACU', 'VILA KOSMOS', 'CAVALCANTI',
       'VILA DA PENHA', 'REALENGO', 'OSWALDO CRUZ', 'ROCHA MIRANDA',
       'CASCADURA', 'BENTO RIBEIRO', 'COLEGIO', 'CIDADE DE DEU

In [17]:
# Verificando as entradas distintas para endereco
escolas.endereco.unique()

array(['RUA CARLOS SEIDL S/No', 'RUA EBANO 187', 'R. DO LAVRADIO, 56',
       'PRACA ARGENTINA 20', 'PRACA CONDESSA PAULO DE FRONTIN 45',
       'R. BARAO DA TORRE, 90', 'RUA EDUARDO SANTOS 38',
       'AVENIDA PRESIDENTE VARGAS S/No', 'RUA ANA NERI 192',
       'R. HADDOCK LOBO, 148', 'RUA VISCONDE DE SILVA S/No',
       'RUA DO CATETE 77', 'AVENIDA OSWALDO CRUZ 124',
       'AV. ATLANTICA, 1976', 'R. BARAO DE IPANEMA, 34',
       'AVENIDA EPITACIO PESSOA 474', 'RUA DA MATRIZ 67',
       'AVENIDA PASTEUR 433', 'RUA ASSUNCAO, 257', 'R. SEN. CORREA',
       'AV HEITOR BELTRAO, S/No', 'R. FERREIRA PONTES, 328',
       'RUA BARAO DE UBA 331', 'R. MAL. JOFRE, 74',
       'BOULEVARD 28 DE SETEMBRO, 353', 'R. ANTONIO BASILIO, 370',
       'RUA ENES DE SOUSA 36', 'R. OITO DE DEZEMBRO, 275',
       'R. ALM. JOAO CANDIDO BRASIL, 352',
       'AVENIDA PRESIDENTE JOAO GOULART 296',
       'RUA MARTAGAO GESTEIRA S/Ndeg', 'R. CIRNE MAIA, 109',
       'RUA HUMBERTO DE CAMPOS S/Ndeg', 'AV. DOM HELDER

Nesse caso, por exemplo, vê-se que existem siglas em escolas_postos:
* "EM", "E.M", "E.M" ou ainda "EM." parecem substituir "ESCOLA MUNICIPAL"
* "CIEP" parece substituir "CENTRO INTEGRADO DE EDUCAÇÃO PUBLICA".

Em bairro, existem alguns casos que contêm a string "MARACANA/ " antes do nome do bairro. Idealmente, teríamos uma lista oficial de bairros para fazer a comparação.


Em endereço, verificam-se os seguintes casos:
* "R." parece substituir "RUA"
* "AV" e "AV." parecem substituir "AVENIDA"
* "PCA." parece substituir "PRACA"
* "ESTR." parece substituir "ESTRADA"
* "S/NO", "S/N.O", "S/N", "S/NDEG", "S/N.DEG" (ESSES 2 ÚLTIMO EU NÃO CONHEÇO) são nomenclaturas que parecem significar "SEM NÚMERO"
* Em alguns casos, como "R. SEN. CORREA" e "R. MAL. JOFRE", existe siglas no nome da rua. Idealmente, teríamos uma base oficial de ruas em bairros para validar o nome da rua, mas como não temos a base nem o conhecimento das ruas no estado em questão, esses casos não serão tratados.
* Alguns têm vírgulas a separar o número do logradouro, outros não

Ressalta-se que só foi possível verificar cada caso porque se trata de uma base de dados pequena. Caso contrário, poderiam ser utilizadas técnicas de amostragem, nuvem de palavras, ou outra técnica para identificar e tratar as siglas.


In [18]:
# Tratamento escolas_postos:
replacements_escolas_postos = {     # '\s' especifica caractere de espaço
    r'^EM\s': 'ESCOLA MUNICIPAL ',     
    r'^E.M\s': 'ESCOLA MUNICIPAL ',  
    r'^EM.\s': 'ESCOLA MUNICIPAL ',   
    r'^E.M.\s': 'ESCOLA MUNICIPAL ',
    r'^CIEP\s': 'CENTRO INTEGRADO DE EDUCACAO PUBLICA ',
}

# Aplicando substituições
for pattern, replacement in replacements_escolas_postos.items():
    escolas.escolas_postos = escolas.escolas_postos.str.replace(pattern, replacement, regex=True)


In [19]:
escolas

,id,escolas_postos,bairro,endereco,lat,lon
0,178,CENTRO INTEGRADO DE EDUCACAO PUBLICA HENFIL,CAJU,RUA CARLOS SEIDL S/No,"-22,880888","-43,225326"
1,634,ESCOLA MUNICIPAL ALICE DO AMARAL PEIXOTO,BENFICA,RUA EBANO 187,"-22,889574","-43,236202"
2,483,ESCOLA MUNICIPAL CELESTINO SILVA,CENTRO,"R. DO LAVRADIO, 56","-22,909293","-43,183579"
3,476,ESCOLA MUNICIPAL FLORIANO PEIXOTO,SAO CRISTOVAO,PRACA ARGENTINA 20,"-22,897629","-43,227456"
4,132,ESCOLA MUNICIPAL PEREIRA PASSOS,RIO COMPRIDO,PRACA CONDESSA PAULO DE FRONTIN 45,"-22,924412","-43,208579"
...,...,...,...,...,...,...
147,474,ESCOLA MUNICIPAL ENGENHEIRO GASTAO RANGEL,GUARATIBA,ESTRADA DO MAGARCA 9.183,"-22,98046","-43,643545"
148,301,ESCOLA MUNICIPAL JONATAS SERRANO,GUARATIBA,"ESTRADA DO MATO ALTO, S/No","-22,953163","-43,577409"
149,215,ESCOLA MUNICIPAL NARCISA AMALIA,ILHA DE GUARATIBA,ESTRADA TEODORETO DE CAMARGO S/N.o,"-23,009084","-43,537582"
150,606,ESCOLA MUNICIPAL PROFESSOR CASTILHO,ILHA DE GUARATIBA,CAMINHO DA MATRIZ 4406,"-22,994124","-43,593683"


In [20]:
escolas.escolas_postos.unique()

array(['CENTRO INTEGRADO DE EDUCACAO PUBLICA HENFIL',
       'ESCOLA MUNICIPAL ALICE DO AMARAL PEIXOTO',
       'ESCOLA MUNICIPAL CELESTINO SILVA',
       'ESCOLA MUNICIPAL FLORIANO PEIXOTO',
       'ESCOLA MUNICIPAL PEREIRA PASSOS',
       'ESCOLA MUNICIPAL PRESIDENTE JOSE LINHARES',
       'ESCOLA MUNICIPAL SANTA CATARINA', 'ESCOLA MUNICIPAL TIA CIATA',
       'ESCOLA MUNICIPAL URUGUAI', 'ESCOLA MUNICIPAL MARIO CLAUDIO',
       'CENTRO INTEGRADO DE EDUCACAO PUBLICA PRESIDENTE AGOSTINHO NETO',
       'CENTRO INTEGRADO DE EDUCACAO PUBLICA TANCREDO NEVES',
       'ESCOLA MUNICIPAL ALBERTO BARTH', 'ESCOLA MUNICIPAL CICERO PENA',
       'ESCOLA MUNICIPAL COCIO BARCELOS',
       'ESCOLA MUNICIPAL HENRIQUE DODSWORTH', 'ESCOLA MUNICIPAL MEXICO',
       'ESCOLA MUNICIPAL MINAS GERAIS',
       'ESCOLA MUNICIPAL PRESIDENTE DA COSTA E SILVA',
       'ESCOLA MUNICIPAL SENADOR CORREA',
       'CENTRO INTEGRADO DE EDUCACAO PUBLICA SAMUEL WEINER',
       'ESCOLA MUNICIPAL AFRANIO PEIXOTO',
       'E

In [21]:
escolas.endereco.unique()

array(['RUA CARLOS SEIDL S/No', 'RUA EBANO 187', 'R. DO LAVRADIO, 56',
       'PRACA ARGENTINA 20', 'PRACA CONDESSA PAULO DE FRONTIN 45',
       'R. BARAO DA TORRE, 90', 'RUA EDUARDO SANTOS 38',
       'AVENIDA PRESIDENTE VARGAS S/No', 'RUA ANA NERI 192',
       'R. HADDOCK LOBO, 148', 'RUA VISCONDE DE SILVA S/No',
       'RUA DO CATETE 77', 'AVENIDA OSWALDO CRUZ 124',
       'AV. ATLANTICA, 1976', 'R. BARAO DE IPANEMA, 34',
       'AVENIDA EPITACIO PESSOA 474', 'RUA DA MATRIZ 67',
       'AVENIDA PASTEUR 433', 'RUA ASSUNCAO, 257', 'R. SEN. CORREA',
       'AV HEITOR BELTRAO, S/No', 'R. FERREIRA PONTES, 328',
       'RUA BARAO DE UBA 331', 'R. MAL. JOFRE, 74',
       'BOULEVARD 28 DE SETEMBRO, 353', 'R. ANTONIO BASILIO, 370',
       'RUA ENES DE SOUSA 36', 'R. OITO DE DEZEMBRO, 275',
       'R. ALM. JOAO CANDIDO BRASIL, 352',
       'AVENIDA PRESIDENTE JOAO GOULART 296',
       'RUA MARTAGAO GESTEIRA S/Ndeg', 'R. CIRNE MAIA, 109',
       'RUA HUMBERTO DE CAMPOS S/Ndeg', 'AV. DOM HELDER

In [22]:
# Tratamento escolas_endereco para abreviações de sem numero:
replacements_escolas_endereco = {
"S/N.o": "",
"S/Ndeg": "",
"S/N.deg": "",
"S/No": "",
"S/NO": "",
"S/N": "",

}
escolas.endereco.replace(replacements_escolas_endereco,regex=True,inplace=True)

In [23]:
# Tratamento escolas_endereco para abreviações de endereço:
replacements_escolas_endereco_ruas_av = {   # '\s' especifica caractere de espaço 
    r'R. ': 'RUA ',              
    r'AV. ': 'AVENIDA ',        
    r'AV\s': 'AVENIDA ',
    r'AV\s': 'AVENIDA ', 
    r"PCA. ": "PRACA ",    
    r"ESTR. " : "ESTRADA ",
}

# Aplicando substituições
for pattern, replacement in replacements_escolas_endereco_ruas_av.items():
    escolas.endereco = escolas.endereco.str.replace(pattern, replacement, regex=True)


In [24]:
escolas.endereco.unique()

array(['RUA CARLOS SEIDL ', 'RUA EBANO 187', 'RUA DO LAVRADIO, 56',
       'PRACA ARGENTINA 20', 'PRACA CONDESSA PAULO DE FRONTIN 45',
       'RUA BARAO DA TORRE, 90', 'RUA EDUARDO SANTOS 38',
       'AVENIDA PRESIDENTE VARGAS ', 'RUA ANA NERUA 192',
       'RUA HADDOCK LOBO, 148', 'RUA VISCONDE DE SILVA ',
       'RUA DO CATETE 77', 'AVENIDA OSWALDO CRUZ 124',
       'AVENIDA ATLANTICA, 1976', 'RUA BARAO DE IPANEMA, 34',
       'AVENIDA EPITACIO PESSOA 474', 'RUA DA MATRIZ 67',
       'AVENIDA PASTEUR 433', 'RUA ASSUNCAO, 257', 'RUA SEN. CORREA',
       'AVENIDA HEITOR BELTRAO, ', 'RUA FERREIRUA PONTES, 328',
       'RUA BARAO DE UBA 331', 'RUA MAL. JOFRE, 74',
       'BOULEVARUA 28 DE SETEMBRO, 353', 'RUA ANTONIO BASILIO, 370',
       'RUA ENES DE SOUSA 36', 'RUA OITO DE DEZEMBRO, 275',
       'RUA ALM. JOAO CANDIDO BRASIL, 352',
       'AVENIDA PRESIDENTE JOAO GOULARUA 296', 'RUA MARTAGAO GESTEIRUA ',
       'RUA CIRNE MAIA, 109', 'RUA HUMBERTO DE CAMPOS ',
       'AVENIDA DOM HELDE

## Tratamento de material_ditatico.csv

In [25]:
df_material_didatico = pd.read_csv("./material_didatico.csv", sep=",")
df_material_didatico.head(10)

,id,Quantidade
0,178,20
1,634,121
2,483,220
3,476,190
4,132,67
5,410,397
6,55,92
7,490,578
8,600,591
9,89,473


In [26]:
df_material_didatico.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151 entries, 0 to 150
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          151 non-null    int64 
 1   Quantidade  147 non-null    object
dtypes: int64(1), object(1)
memory usage: 2.5+ KB


In [27]:
df_material_didatico["id"] = df_material_didatico.id.astype(str)

# Inclui zeros até que a string tenha tamanho 3
df_material_didatico["id"] = df_material_didatico.id.str.zfill(3)

In [28]:
df_material_didatico.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151 entries, 0 to 150
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          151 non-null    object
 1   Quantidade  147 non-null    object
dtypes: object(2)
memory usage: 2.5+ KB


In [29]:
df_material_didatico.head(10)

,id,Quantidade
0,178,20
1,634,121
2,483,220
3,476,190
4,132,67
5,410,397
6,055,92
7,490,578
8,600,591
9,089,473


In [30]:
df_material_didatico.columns = df_material_didatico.columns.str.lower()
df_material_didatico.columns

Index(['id', 'quantidade'], dtype='object')

In [31]:
# nova coluna 'Quantidade_original' com os valores originais para manter histórico
df_material_didatico['quantidade_original'] = df_material_didatico['quantidade']

# retirando possiveis valores com aspas
df_material_didatico['quantidade'] = df_material_didatico['quantidade'].str.replace("'", '')

# convertendo para numerico / NaN para valores não numéricos
df_material_didatico['quantidade'] = pd.to_numeric(df_material_didatico['quantidade'], errors='coerce')

## Tratamento de subprefeituras.csv

In [32]:
df_subprefeituras = pd.read_csv("./subprefeituras.csv", sep=",")
df_subprefeituras

,nome,subprefeitura
0,Santo Cristo,Centro
1,Gamboa,Centro
2,Saúde,Centro
3,Caju,Centro
4,Centro,Centro
...,...,...
159,Santa Cruz,Zona Oeste
160,Guaratiba,Zona Oeste
161,Barra de Guaratiba,Zona Oeste
162,Pedra de Guaratiba,Zona Oeste


In [33]:
trata_string(df_subprefeituras,"nome", "subprefeitura")

,nome,subprefeitura
0,SANTO CRISTO,CENTRO
1,GAMBOA,CENTRO
2,SAUDE,CENTRO
3,CAJU,CENTRO
4,CENTRO,CENTRO
...,...,...
159,SANTA CRUZ,ZONA OESTE
160,GUARATIBA,ZONA OESTE
161,BARRA DE GUARATIBA,ZONA OESTE
162,PEDRA DE GUARATIBA,ZONA OESTE


## One BigTable

In [34]:
bigtable = pd.merge(escolas, df_material_didatico, on="id", how="left")
bigtable.head(10)

,id,escolas_postos,bairro,endereco,lat,lon,quantidade,quantidade_original
0,178,CENTRO INTEGRADO DE EDUCACAO PUBLICA HENFIL,CAJU,RUA CARLOS SEIDL,"-22,880888","-43,225326",20.0,20
1,634,ESCOLA MUNICIPAL ALICE DO AMARAL PEIXOTO,BENFICA,RUA EBANO 187,"-22,889574","-43,236202",121.0,121
2,483,ESCOLA MUNICIPAL CELESTINO SILVA,CENTRO,"RUA DO LAVRADIO, 56","-22,909293","-43,183579",220.0,220
3,476,ESCOLA MUNICIPAL FLORIANO PEIXOTO,SAO CRISTOVAO,PRACA ARGENTINA 20,"-22,897629","-43,227456",190.0,190
4,132,ESCOLA MUNICIPAL PEREIRA PASSOS,RIO COMPRIDO,PRACA CONDESSA PAULO DE FRONTIN 45,"-22,924412","-43,208579",67.0,67
5,017,ESCOLA MUNICIPAL PRESIDENTE JOSE LINHARES,IPANEMA,"RUA BARAO DA TORRE, 90","-22,983332","-43,199036",48.0,48
6,055,ESCOLA MUNICIPAL SANTA CATARINA,SANTA TERESA,RUA EDUARDO SANTOS 38,"-22,915535","-43,191588",92.0,92
7,490,ESCOLA MUNICIPAL TIA CIATA,CENTRO,AVENIDA PRESIDENTE VARGAS,"-22,907123","-43,195068",578.0,578
8,600,ESCOLA MUNICIPAL URUGUAI,BENFICA,RUA ANA NERUA 192,"-22,898488","-43,237756",591.0,591
9,089,ESCOLA MUNICIPAL MARIO CLAUDIO,RIO COMPRIDO,"RUA HADDOCK LOBO, 148","-22,917233","-43,211044",473.0,473


In [35]:
bigtable = pd.merge(bigtable, df_subprefeituras, left_on="bairro", right_on="nome", how="left")
bigtable.head(10)

,id,escolas_postos,bairro,endereco,lat,lon,quantidade,quantidade_original,nome,subprefeitura
0,178,CENTRO INTEGRADO DE EDUCACAO PUBLICA HENFIL,CAJU,RUA CARLOS SEIDL,"-22,880888","-43,225326",20.0,20,CAJU,CENTRO
1,634,ESCOLA MUNICIPAL ALICE DO AMARAL PEIXOTO,BENFICA,RUA EBANO 187,"-22,889574","-43,236202",121.0,121,BENFICA,CENTRO
2,483,ESCOLA MUNICIPAL CELESTINO SILVA,CENTRO,"RUA DO LAVRADIO, 56","-22,909293","-43,183579",220.0,220,CENTRO,CENTRO
3,476,ESCOLA MUNICIPAL FLORIANO PEIXOTO,SAO CRISTOVAO,PRACA ARGENTINA 20,"-22,897629","-43,227456",190.0,190,SAO CRISTOVAO,CENTRO
4,132,ESCOLA MUNICIPAL PEREIRA PASSOS,RIO COMPRIDO,PRACA CONDESSA PAULO DE FRONTIN 45,"-22,924412","-43,208579",67.0,67,RIO COMPRIDO,CENTRO
5,017,ESCOLA MUNICIPAL PRESIDENTE JOSE LINHARES,IPANEMA,"RUA BARAO DA TORRE, 90","-22,983332","-43,199036",48.0,48,IPANEMA,ZONA SUL
6,055,ESCOLA MUNICIPAL SANTA CATARINA,SANTA TERESA,RUA EDUARDO SANTOS 38,"-22,915535","-43,191588",92.0,92,SANTA TERESA,CENTRO
7,490,ESCOLA MUNICIPAL TIA CIATA,CENTRO,AVENIDA PRESIDENTE VARGAS,"-22,907123","-43,195068",578.0,578,CENTRO,CENTRO
8,600,ESCOLA MUNICIPAL URUGUAI,BENFICA,RUA ANA NERUA 192,"-22,898488","-43,237756",591.0,591,BENFICA,CENTRO
9,089,ESCOLA MUNICIPAL MARIO CLAUDIO,RIO COMPRIDO,"RUA HADDOCK LOBO, 148","-22,917233","-43,211044",473.0,473,RIO COMPRIDO,CENTRO


In [36]:
# agrupando por subprefeitura e fazendo uma media da quantidade de material didatico
media_por_subprefeitura = bigtable.groupby("subprefeitura").quantidade.mean().round().astype(int)
media_por_subprefeitura

subprefeitura
BARRA DA TIJUCA    108
CENTRO             314
GRANDE BANGU       294
GRANDE TIJUCA      304
ILHAS              370
JACAREPAGUA        352
ZONA NORTE         325
ZONA OESTE         321
ZONA SUL           307
Name: quantidade, dtype: int64

In [37]:
# valores com NaN
bigtable[bigtable.quantidade.isna()]

,id,escolas_postos,bairro,endereco,lat,lon,quantidade,quantidade_original,nome,subprefeitura
14,413,ESCOLA MUNICIPAL COCIO BARCELOS,COPACABANA,"RUA BARAO DE IPANEMA, 34","-22,974828","-43,189063",NaN,'',COPACABANA,ZONA SUL
15,413,ESCOLA MUNICIPAL COCIO BARCELOS,COPACABANA,"RUA BARAO DE IPANEMA, 34","-22,74828","-43,189063",NaN,'',COPACABANA,ZONA SUL
31,060,ESCOLA MUNICIPAL BARAO HOMEM DE MELO,MARACANA/ TIJUCA,"RUA ALM. JOAO CANDIDO BRASIL, 352","-22,916829","-43,236925",NaN,'NaN',NaN,NaN
38,429,ESCOLA MUNICIPAL FELIX PACHECO,PIEDADE,"RUA ASSIS CARNEIRO, 649","-22,895086","-43,313826",NaN,NaN,PIEDADE,ZONA NORTE
77,091,ESCOLA MUNICIPAL PIAUI,GUADALUPE,AVENIDA BRASIL 23.364,"-22,845415","-43,37852",NaN,'NaN',GUADALUPE,ZONA NORTE
100,492,ESCOLA MUNICIPAL MENEZES CORTES,FREGUESIA JPA,PRACA JOSE ALVES DE AZEVEDO 43,"-22,942655","-43,338969",NaN,NaN,NaN,NaN
118,687,ESCOLA MUNICIPAL TASSO DA SILVEIRA,REALENGO,RUA GENERAL BERNARDINO DE MATOS,"-22,883859","-43,41733",NaN,NaN,REALENGO,GRANDE BANGU
141,297,ESCOLA MUNICIPAL PROFESSORA SILVIA DE ARAUJO T...,SANTA CRUZ,AVENIDA CANAL 3 CONJ. CESARAO,"-22,930111","-43,652899",NaN,' ',SANTA CRUZ,ZONA OESTE
149,215,ESCOLA MUNICIPAL NARCISA AMALIA,ILHA DE GUARATIBA,ESTRADA TEODORETO DE CAMARGO,"-23,009084","-43,537582",NaN,NaN,ILHA DE GUARATIBA,ZONA OESTE


In [38]:
# aplicando a media da subprefeitura de quantidade de material didatico na coluna quantidade
bigtable['quantidade'] = bigtable['quantidade'].fillna(bigtable['subprefeitura'].map(media_por_subprefeitura))


In [39]:
bigtable.head(10)

,id,escolas_postos,bairro,endereco,lat,lon,quantidade,quantidade_original,nome,subprefeitura
0,178,CENTRO INTEGRADO DE EDUCACAO PUBLICA HENFIL,CAJU,RUA CARLOS SEIDL,"-22,880888","-43,225326",20.0,20,CAJU,CENTRO
1,634,ESCOLA MUNICIPAL ALICE DO AMARAL PEIXOTO,BENFICA,RUA EBANO 187,"-22,889574","-43,236202",121.0,121,BENFICA,CENTRO
2,483,ESCOLA MUNICIPAL CELESTINO SILVA,CENTRO,"RUA DO LAVRADIO, 56","-22,909293","-43,183579",220.0,220,CENTRO,CENTRO
3,476,ESCOLA MUNICIPAL FLORIANO PEIXOTO,SAO CRISTOVAO,PRACA ARGENTINA 20,"-22,897629","-43,227456",190.0,190,SAO CRISTOVAO,CENTRO
4,132,ESCOLA MUNICIPAL PEREIRA PASSOS,RIO COMPRIDO,PRACA CONDESSA PAULO DE FRONTIN 45,"-22,924412","-43,208579",67.0,67,RIO COMPRIDO,CENTRO
5,017,ESCOLA MUNICIPAL PRESIDENTE JOSE LINHARES,IPANEMA,"RUA BARAO DA TORRE, 90","-22,983332","-43,199036",48.0,48,IPANEMA,ZONA SUL
6,055,ESCOLA MUNICIPAL SANTA CATARINA,SANTA TERESA,RUA EDUARDO SANTOS 38,"-22,915535","-43,191588",92.0,92,SANTA TERESA,CENTRO
7,490,ESCOLA MUNICIPAL TIA CIATA,CENTRO,AVENIDA PRESIDENTE VARGAS,"-22,907123","-43,195068",578.0,578,CENTRO,CENTRO
8,600,ESCOLA MUNICIPAL URUGUAI,BENFICA,RUA ANA NERUA 192,"-22,898488","-43,237756",591.0,591,BENFICA,CENTRO
9,089,ESCOLA MUNICIPAL MARIO CLAUDIO,RIO COMPRIDO,"RUA HADDOCK LOBO, 148","-22,917233","-43,211044",473.0,473,RIO COMPRIDO,CENTRO


In [40]:
# valores que estão com NaN em quantidade
bigtable[bigtable.quantidade.isna()]

,id,escolas_postos,bairro,endereco,lat,lon,quantidade,quantidade_original,nome,subprefeitura
31,060,ESCOLA MUNICIPAL BARAO HOMEM DE MELO,MARACANA/ TIJUCA,"RUA ALM. JOAO CANDIDO BRASIL, 352","-22,916829","-43,236925",NaN,'NaN',NaN,NaN
100,492,ESCOLA MUNICIPAL MENEZES CORTES,FREGUESIA JPA,PRACA JOSE ALVES DE AZEVEDO 43,"-22,942655","-43,338969",NaN,NaN,NaN,NaN


In [41]:
bigtable

,id,escolas_postos,bairro,endereco,lat,lon,quantidade,quantidade_original,nome,subprefeitura
0,178,CENTRO INTEGRADO DE EDUCACAO PUBLICA HENFIL,CAJU,RUA CARLOS SEIDL,"-22,880888","-43,225326",20.0,20,CAJU,CENTRO
1,634,ESCOLA MUNICIPAL ALICE DO AMARAL PEIXOTO,BENFICA,RUA EBANO 187,"-22,889574","-43,236202",121.0,121,BENFICA,CENTRO
2,483,ESCOLA MUNICIPAL CELESTINO SILVA,CENTRO,"RUA DO LAVRADIO, 56","-22,909293","-43,183579",220.0,220,CENTRO,CENTRO
3,476,ESCOLA MUNICIPAL FLORIANO PEIXOTO,SAO CRISTOVAO,PRACA ARGENTINA 20,"-22,897629","-43,227456",190.0,190,SAO CRISTOVAO,CENTRO
4,132,ESCOLA MUNICIPAL PEREIRA PASSOS,RIO COMPRIDO,PRACA CONDESSA PAULO DE FRONTIN 45,"-22,924412","-43,208579",67.0,67,RIO COMPRIDO,CENTRO
...,...,...,...,...,...,...,...,...,...,...
147,474,ESCOLA MUNICIPAL ENGENHEIRO GASTAO RANGEL,GUARATIBA,ESTRADA DO MAGARCA 9.183,"-22,98046","-43,643545",320.0,320,GUARATIBA,ZONA OESTE
148,301,ESCOLA MUNICIPAL JONATAS SERRANO,GUARATIBA,"ESTRADA DO MATO ALTO,","-22,953163","-43,577409",335.0,335,GUARATIBA,ZONA OESTE
149,215,ESCOLA MUNICIPAL NARCISA AMALIA,ILHA DE GUARATIBA,ESTRADA TEODORETO DE CAMARGO,"-23,009084","-43,537582",321.0,NaN,ILHA DE GUARATIBA,ZONA OESTE
150,606,ESCOLA MUNICIPAL PROFESSOR CASTILHO,ILHA DE GUARATIBA,CAMINHO DA MATRIZ 4406,"-22,994124","-43,593683",521.0,521,ILHA DE GUARATIBA,ZONA OESTE


----

### possui id no material_didatico.csv mas não no escolas.csv

In [42]:
ids_faltantes = df_material_didatico[~df_material_didatico['id'].isin(bigtable['id'])]
ids_faltantes

,id,quantidade,quantidade_original
5,410,397.0,397
15,698,366.0,366
25,271,152.0,152
40,367,297.0,297
49,647,64.0,64
56,683,218.0,218
68,236,41.0,41
77,285,32.0,32
79,441,338.0,338
83,182,425.0,425


## Criação dos bancos finais

## Desafio